In [2]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import calendar
import undetected_chromedriver as uc
import random

## Pages Extraction - Ciper

In [ ]:

json_pages_info = {"pages": []}

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

articles_count = 0

pages = 100

url = "https://www.ciperchile.cl/page/1/?s=Estallido+Social"

driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(2)

with open(f"output.html", "w", encoding="utf-8") as f:
		f.write(driver.page_source)

for page in range(2, pages + 1):

	try:
		elements = driver.find_element(By.CLASS_NAME, "pb-4")
		articles = elements.find_elements(By.CLASS_NAME, "col-md-12")
	except Exception as e:
		print("Fatal error al encontrar los elementos de la página: ", e)
		break

	print(f"{len(articles)} articles found")

	for article in articles:
		print("Analyzing article...")

		date = article.find_element(By.CLASS_NAME, "article__text--date")
		print(date.text)
		originalDate = date.text[:10] if date else "not found"
		splitDate = originalDate.split("-")

		if(len(splitDate) != 3 or date == None):
			print("Fecha no encontrada")
			continue
		
		timestamp = calendar.timegm(time.strptime(f'{splitDate[2]}-{splitDate[1]}-{splitDate[0]} 00:00:00', '%Y-%m-%d %H:%M:%S'))

		# print("timestamp: " + str(timestamp))

		if (timestamp < 1573786800 or timestamp > 1702782000):
			print("La noticia no corresponde a la fecha solicitada")
			continue

		print(splitDate)

		try:
			link = article.find_elements(By.CLASS_NAME, "alticle-link")[1].get_attribute("href")
		except Exception as e:
			print("Error al obtener el enlace del artículo: ", e)
			continue

		try:
			category = article.find_element(By.CLASS_NAME, "alticle-link").text
			title = article.find_element(By.CLASS_NAME, "article__text--title").text
			description = article.find_element(By.CLASS_NAME, "article__text--epigraph").text

			link_info = {
				"newscast" : "Ciper",
				"title": title,
				"description": description,
				"category": category,
				"date": originalDate,
				"image_link": "The site does not provide a image",
				"author": "not found initially",
				"link": link,
			}

		except Exception as e:
			print("Error al obtener los datos del artículo: ", e)

			link_info = {
				"newscast": "Error",
				"title": "Error",
				"description": "Error",
				"category": "Error",
				"date": originalDate,
				"image_link": "Error",
				"author": "Error",
				"link": link
			}

		articles_count += 1
		print("Añadiendo articulo")
		json_pages_info["pages"].append(link_info)

	url = f"https://www.ciperchile.cl/page/{page}/?s=Estallido+Social"
	driver.get(url)
	time.sleep(2)

driver.quit()

with open(f"../archive/temp/ciper_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


## Content Extraction - Ciper

In [11]:

extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}

with open("../archive/pages_extracted/ciper/ciper_090725_cleanData.json", 'r', encoding='utf-8') as file:
	extracted_pages = json.load(file)

data_extracted = True
re = False

for page_index, page in enumerate(extracted_pages["pages"]):

	print(f"Link - {page['link']}" )
	url = page['link']
	first = True

	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			try: 
				title = driver.find_element(By.TAG_NAME, "h1").text
				content = driver.find_element(By.CLASS_NAME, "col-lg-9").find_element(By.CLASS_NAME, "col-lg-9").text
				page["content"] = content
				extracted_pages_with_content["pages"].append(page)
				print(title)
			except Exception as e:
				print(f"Error al procesar el artículo: {e}")
				continue

		except Exception as e:
			print(f"Error al procesar. Página: {page_index}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

	print("Almacenando información en el archivo JSON...")
	with open(f"../archive/temp/ciper_pages_with_content.json", 'w', encoding='utf-8') as file:
		json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

	driver.quit()
	time.sleep(random.uniform(6, 12))

Link - https://www.ciperchile.cl/2022/12/21/reforma-previsional-sin-abusar/
CARTAS: Necesitamos una reforma previsional, pero sin abusar de pymes y medianas empresas
Almacenando información en el archivo JSON...
Link - https://www.ciperchile.cl/2022/11/23/cartas-proposito-del-simce/
CARTAS: ¿Cuál es el propósito del SIMCE en 2022?
Almacenando información en el archivo JSON...
Link - https://www.ciperchile.cl/2022/10/26/cartas-reforma-a-carabineros-y-responsabilidad-de-altos-mandos/
CARTAS: Reforma a Carabineros y responsabilidad de altos mandos
Almacenando información en el archivo JSON...
Link - https://www.ciperchile.cl/2022/09/30/ni-pueblo-ni-elites-otro-diagnostico-del-estallido-social-al-rechazo/
Ni «pueblo» ni «élites»: otro diagnóstico del estallido social al Rechazo
Almacenando información en el archivo JSON...
Link - https://www.ciperchile.cl/2022/03/09/balance-de-un-periodo-2-el-traspaso-de-una-banda/
11M: El traspaso de una banda presidencial deshilachada
Almacenando informa